## Lab 01. Poetry generation

Let's try to generate some poetry using RNNs. 

You have several choices here: 

* The Shakespeare sonnets, file `sonnets.txt` available in the notebook directory.

* Роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина. В предобработанном виде доступен по [ссылке](https://github.com/attatrol/data_sources/blob/master/onegin.txt).

* Some other text source, if it will be approved by the course staff.

Text generation can be designed in several steps:
    
1. Data loading.
2. Dictionary generation.
3. Data preprocessing.
4. Model (neural network) training.
5. Text generation (model evaluation).


In [2]:
import string
import os
import numpy as np

import torch, torch.nn as nn
import torch.nn.functional as F

### Data loading: "Евгений Онегин"


In [3]:
!wget https://raw.githubusercontent.com/attatrol/data_sources/master/onegin.txt
    
with open('onegin.txt', 'r', encoding="utf8") as iofile:
    text = iofile.readlines()
    
text = [x.replace('\t\t', '') for x in text]

--2021-04-27 13:15:25--  https://raw.githubusercontent.com/attatrol/data_sources/master/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: 'onegin.txt.4'

     0K .......... .......... .......... .......... .......... 19%  488K 0s
    50K .......... .......... .......... .......... .......... 39%  784K 0s
   100K .......... .......... .......... .......... .......... 58%  184K 0s
   150K .......... .......... .......... .......... .......... 78% 1,69M 0s
   200K .......... .......... .......... .......... .......... 97% 22,5M 0s
   250K ......                                                100% 23,9M=0,5s

2021-04-27 13:15:26 (547 KB/s) - 'onegin.txt.4' saved [262521/262521]



In opposite to the in-class practice, this time we want to predict complex text. Let's reduce the complexity of the task and lowercase all the symbols.

Now variable `text` is a list of strings. Join all the strings into one and lowercase it.

In [4]:
# Join all the strings into one and lowercase it
# Put result into variable text.

out = ''.join(text).lower()

Put all the characters, that you've seen in the text, into variable `tokens`.

In [5]:
tokens = sorted(set(out))

Create dictionary `token_to_idx = {<char>: <index>}` and dictionary `idx_to_token = {<index>: <char>}`

In [6]:
# dict <index>:<char>
index_to_token = {
    idx: token
    for idx, token  in enumerate(tokens)
}

# dict <char>:<index>
token_to_index = {
    token: idx
    for idx, token  in enumerate(tokens)
}

*Comment: in this task we have only 38 different tokens, so let's use one-hot encoding.*

In [7]:
def to_matrix(texts, max_len=None, pad=token_to_index[' '], dtype='int32', batch_first=True):
    
    max_len = max_len or max(map(len, texts))
    texts_ix = np.zeros([len(texts), max_len], dtype) + pad

    for i in range(len(texts)):
        line_ix = [token_to_index[c] for c in texts[i]]
        texts_ix[i, :len(line_ix)] = line_ix[:max_len]
        
    if not batch_first:
        texts_ix = np.transpose(names_ix)

    return texts_ix

### Building the model

Now we want to build and train recurrent neural net which would be able to something similar to Shakespeare's poetry.

Let's use vanilla RNN, similar to the one created during the lesson.

In [8]:
class CharRNNCell(nn.Module):
    """
    Implement the scheme above as torch module
    """
    def __init__(self, num_tokens=len(tokens), embedding_size=16, rnn_num_units=64):
        super(self.__class__,self).__init__()
        self.num_units = rnn_num_units
        
        self.embedding = nn.Embedding(num_tokens, embedding_size)
        self.rnn_update = nn.Linear(
            embedding_size + rnn_num_units,
            rnn_num_units
        )
        self.rnn_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x, h_prev):
        """
        This method computes h_next(x, h_prev) and log P(x_next | h_next)
        We'll call it repeatedly to produce the whole sequence.
        
        :param x: batch of character ids, containing vector of int64
        :param h_prev: previous rnn hidden states, containing matrix [batch, rnn_num_units] of float32
        """
        # get vector embedding of x
        x_emb = self.embedding(x)
        
        # compute next hidden state using self.rnn_update
        # hint: use torch.cat(..., dim=...) for concatenation
        x_and_h = torch.cat([x_emb, h_prev], dim=-1)# YOUR CODE HERE
        h_next = self.rnn_update(x_and_h) # YOUR CODE HERE
        
        h_next = torch.tanh(h_next) # YOUR CODE HERE
        
        assert h_next.size() == h_prev.size()
        
        #compute logits for next character probs
        logits = self.rnn_to_logits(h_next) # YOUR CODE
        
        return h_next, F.log_softmax(logits, -1)
    
    def initial_state(self, batch_size):
        """ return rnn state before it processes first input (aka h0) """
        return torch.zeros(batch_size, self.num_units, requires_grad=True)

In [9]:
def rnn_loop(char_rnn, batch_ix):
    """
    Computes log P(next_character) for all time-steps in names_ix
    :param names_ix: an int32 matrix of shape [batch, time], output of to_matrix(names)
    """
    batch_size, max_length = batch_ix.size()
    hid_state = char_rnn.initial_state(batch_size)
    logprobs = []

    for x_t in batch_ix.transpose(0,1):
        hid_state, logp_next = char_rnn(x_t, hid_state)  # <-- here we call your one-step code
        logprobs.append(logp_next)
        
    return torch.stack(logprobs, dim=1)

In [10]:
import matplotlib.pyplot as plt

In [11]:
from IPython.display import clear_output
from random import sample

char_rnn = CharRNNCell()
criterion = nn.NLLLoss()
opt = torch.optim.Adam(char_rnn.parameters())
history = []

In [12]:
names = out.split('\n')
names = [elem for elem in names if elem != '']

In [13]:
names

['i',
 '«мой дядя самых честных правил,',
 'когда не в шутку занемог,',
 'он уважать себя заставил',
 'и лучше выдумать не мог.',
 'его пример другим наука;',
 'но, боже мой, какая скука',
 'с больным сидеть и день и ночь,',
 'не отходя ни шагу прочь!',
 'какое низкое коварство',
 'полуживого забавлять,',
 'ему подушки поправлять,',
 'печально подносить лекарство,',
 'вздыхать и думать про себя:',
 'когда же черт возьмет тебя!»',
 'ii',
 'так думал молодой повеса,',
 'летя в пыли на почтовых,',
 'всевышней волею зевеса',
 'наследник всех своих родных. —',
 'друзья людмилы и руслана!',
 'с героем моего романа',
 'без предисловий, сей же час',
 'позвольте познакомить вас:',
 'онегин, добрый мой приятель,',
 'родился на брегах невы,',
 'где, может быть, родились вы',
 'или блистали, мой читатель;',
 'там некогда гулял и я:',
 'но вреден север для меня',
 'iii',
 'служив отлично-благородно,',
 'долгами жил его отец,',
 'давал три бала ежегодно',
 'и промотался наконец.',
 'судьба евгения х

In [14]:
np.mean(list(map(len, names)))

24.00712377560107

In [20]:
num_tokens = 83

In [15]:
batch_ix = to_matrix(sample(names, 32), max_len=30)

In [17]:
batch_ix = torch.tensor(batch_ix, dtype=torch.int64)

In [18]:
logp_seq = rnn_loop(char_rnn, batch_ix)

In [21]:
predictions_logp = logp_seq[:, :-1]
actual_next_tokens = batch_ix[:, 1:]
loss = criterion(predictions_logp.contiguous().view(-1, num_tokens), 
              actual_next_tokens.contiguous().view(-1))

In [22]:
# train with backprop
loss.backward()
opt.step()
opt.zero_grad()

In [23]:
history.append(loss.data.numpy())

In [24]:
history

[array(4.5062265, dtype=float32)]

In [ ]:
MAX_LENGTH = 30

for i in range(1000):
    batch_ix = to_matrix(sample(names, 32), max_len=MAX_LENGTH)
    batch_ix = torch.tensor(batch_ix, dtype=torch.int64)
    
    logp_seq = rnn_loop(char_rnn, batch_ix)
    
    # compute loss
    predictions_logp = logp_seq[:, :-1]
    actual_next_tokens = batch_ix[:, 1:]
    loss = criterion(predictions_logp.contiguous().view(-1, num_tokens), 
                  actual_next_tokens.contiguous().view(-1))
    
    # train with backprop
    loss.backward()
    opt.step()
    opt.zero_grad()
    # YOUR CODE HERE
    
    history.append(loss.data.numpy())
    if (i+1)%100==0:
        clear_output(True)
        plt.plot(history,label='loss')
        plt.legend()
        plt.show()

In [25]:
assert np.mean(history[:10]) > np.mean(history[-10:]), "RNN didn't converge."

AssertionError: RNN didn't converge.

Plot the loss function (axis X: number of epochs, axis Y: loss function).

In [3]:
# Your plot code here

In [ ]:
def generate_sample(char_rnn, seed_phrase=' Hello', max_length=MAX_LENGTH, temperature=1.0):
    '''
    ### Disclaimer: this is an example function for text generation.
    ### You can either adapt it in your code or create your own function
    
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs, 
        smaller temperature converges to the single most likely output.
        
    Be careful with the model output. This model waits logits (not probabilities/log-probabilities)
    of the next symbol.
    '''
    
    x_sequence = [token_to_id[token] for token in seed_phrase]
    x_sequence = torch.tensor([[x_sequence]], dtype=torch.int64)
    hid_state = char_rnn.initial_state(batch_size=1)
    
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        print(x_sequence[:, -1].shape, hid_state.shape)
        out, hid_state = char_rnn(x_sequence[:, i], hid_state)
    
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        print(x_sequence.shape, x_sequence, hid_state.shape)
        out, hid_state = char_rnn(x_sequence[:, -1], hid_state)
        # Be really careful here with the model output
        p_next = F.softmax(out / temperature, dim=-1).data.numpy()[0]
        
        # sample next token and push it back into x_sequence
        print(p_next.shape, len(tokens))
        next_ix = np.random.choice(len(tokens), p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64)
        print(x_sequence.shape, next_ix.shape)
        x_sequence = torch.cat([x_sequence, next_ix], dim=1)
        
    return ''.join([tokens[ix] for ix in x_sequence.data.numpy()[0]])

In [40]:
# An example of generated text.
# print(generate_text(length=500, temperature=0.2))

hide my will in thine?
  shall will in of the simend that in my sime the seave the seave the sorll the soren the sange the seall seares and and the fart the wirl the seall the songh whing that thou hall will thoun the soond beare the with that sare the simest me the fart the wirl the songre the with thy seart so for shat so for do the dost the sing the sing the sing the soond canding the sack and the farling the wirl of sore sich and that with the seare the seall so fort the with the past the wirl the simen the wirl the sores the sare


### More poetic model

Let's use LSTM instead of vanilla RNN and compare the results.

Plot the loss function of the number of epochs. Does the final loss become better?

In [46]:
# Your beautiful code here

Generate text using the trained net with different `temperature` parameter: `[0.1, 0.2, 0.5, 1.0, 2.0]`.

Evaluate the results visually, try to interpret them.

In [47]:
# Text generation with different temperature values here

### Saving and loading models

Save the model to the disk, then load it and generate text. Examples are available [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html]).

In [4]:
# Saving and loading code here

### References
1. <a href='http://karpathy.github.io/2015/05/21/rnn-effectiveness/'> Andrew Karpathy blog post about RNN. </a> 
There are several examples of genration: Shakespeare texts, Latex formulas, Linux Sourse Code and children names.
2. <a href='https://github.com/karpathy/char-rnn'> Repo with char-rnn code </a>
3. Cool repo with PyTorch examples: [link](https://github.com/spro/practical-pytorch`)